In [44]:
import requests
from bs4 import BeautifulSoup

res = []
ress = []
for genre in range(1, 2):
    for page in range(1,3):
        url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id={}&page={}'.format(genre,page)
        #print(url)
        r = requests.get(url)
        soup = BeautifulSoup(r.text)

        infos = []
        imgs = []
        for d in soup.find_all('div', class_="release_info"):
            info = {}
            info['中文名稱'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').text.strip()
            info['英文名稱'] = d.find('div', class_='release_movie_name').find('div', class_='en').a.text.strip()
            info['上映日期'] = d.find('div', class_='release_movie_time').text.split('上映日期 ： ')[1]
            infos.append(info)
            
        for d in soup.find_all('div', class_='release_foto'):
            img = {}
            img['封面照片網址'] = d.find('img')['src']
            imgs.append(img)

        #for i, j in list(zip(infos, imgs)):
            #res.append(dict(list(i.items()) + list(j.items())))
        for inf in list(infos):
            ress.append(dict(list(inf.items())))



TypeError: not enough arguments for format string

In [43]:
import pandas as pd

df = pd.DataFrame(ress)
df.to_csv('movies.csv', encoding='utf_8_sig')

df

,中文名稱,英文名稱,上映日期
0,小小兵2：格魯的崛起,Minions: The Rise of Guru,2022-07-01
1,蝙蝠俠,The Batman,2022-03-04
2,魔比斯,Morbius,2022-01-16
3,捍衛戰士：獨行俠,Top Gun: Maverick,2021-11-19
4,魔鬼剋星：未來世,Ghostbusters: Afterlife,2021-11-11
...,...,...,...
1262,劇場版 假面騎士ZERO-ONE REAL×TIME+劇場短篇 假面騎士聖刃 不死鳥的劍士與...,,未定
1263,讀報人,News of the World,未定
1264,沉默的證人,Bodies At Rest,未定
1265,狄仁傑之四大天王,Detective Dee The Four Heavenly Kings,未定
